In [1]:
from lib.tools.metadata import get_info, valid_attr
from lib.tools.utils import is_dir, crop_image
from config.config import SEP_SYMBOL_COLM, SEP_SYMBOL_NAME, N_PERIOD_RETURN
from config.charts import PlotConfig

# 0. Vista general del proyecto


In [2]:
# Abreviación del proyecto
project_name = "Tia-Maria"

# Crear ruta de proyecto
output_dir = f"output/{project_name}/"

# Crear ruta de salida
is_dir(output_dir)

True

In [3]:
# Configuración de formatos de reportes
PlotConfig.setup_matplotlib()

# 1. Identificación de fallas


In [4]:
from lib.faults.faults import FaultIdentifier
from lib.plotters.maps import Map
from lib.tools.utils import create_circle, calc_extent

import geopandas as gpd

## 1.1 Datos de entrada


In [5]:
# Identificar fallas dentro de un radio específico
longitude, latitude, radius = -77.411534, -8.060878, 100

# Factor de escala de longitud de falla en columna del SHP
scale_factor = 100

# Modelo de fallas geológicas
key_faults = "INGEMMET (2023)"

# Identificador de fallas del modelo
fault_names = "NOMBRE"

## 1.2 Inicialización


In [6]:
# Crear ruta de salida del módulo
output_module = f"{output_dir}/01_fault_identification/"
is_dir(output_module)

# Nombre de archivo de salida
disagg_file = f"{output_module}/01_faults_table"
faults_map = f"{output_module}/01_faults_map"

In [7]:
# Inicializar identificador de fallas
fault_identifier = FaultIdentifier.known_shp(key=key_faults)

print(
    f"""Atributos de metadata válidos:
      {valid_attr(fault_identifier)}"""
)


faults_in = fault_identifier.within(longitude, latitude, radius)

Atributos de metadata válidos:
      ['circle_params', 'columns_shp', 'metadata']


In [8]:
# Verificar atributos del archivo SHP
# print(get_info(fault_identifier, 'columns_shp'))

## 1.3 Procesamiento


In [9]:
# Si se encontraron geometrías, analizarlas con agregaciones personalizadas
if not faults_in.empty:
    # Diccionario de columnas y funciones de agregación
    agg_dict = {
        "OBJECTID": "first",
        "CINEMATICA": "first",
        "BUZAMIENTO": "first",
        "DIR_BUZ": "first",
        "TASA_MOV": "first",
        "COD_GEOCAT": "first",
        "EDAD_ULT_M": "first",
        "SHAPE_LEN": lambda x: x.sum() * scale_factor,
    }

    faults_df = fault_identifier.get_faults(faults_in, fault_names, agg_dict)
else:
    print("No se encontraron fallas en el área especificada.")

## 1.4 Creación de mapa


In [10]:
# Calcular la extensión del mapa
extent = calc_extent(latitude, longitude, side_length=400)

# Crear el círculo
circle = create_circle((longitude, latitude), radius)

# Crear el GeoDataFrame
circle_gdf = gpd.GeoDataFrame({"geometry": [circle]}, crs="EPSG:4326")
circle_gdf["name"] = ["Radio de análisis"]

In [11]:
base_map = Map(figsize=(10, 10), extent=extent)
base_map.add_features(linestyle="-", linewidth=0.5, color="gray")
base_map.add_sat_img(zoom=10, alpha=1, attribution_size=5)

# Añadir datos del gdf a la instancia de Map
base_map.add_gdf(faults_in, column=fault_names, linewidth=3, alpha=1)

# Añadir gdf del radio de análisis
base_map.add_gdf(circle_gdf, column="name", color_map="Greys")

# Añadir punto de análisis del proyecto
base_map.add_scatter(
    longitude,
    latitude,
    facecolor="red",
    edgecolor=None,
    marker="*",
    s=250,
    label="Punto de análisis",
)

# Limpiar la leyenda actual y volver a crearla con entradas únicas
base_map.add_legend(framealpha=1)

# Añadir detalles
base_map.add_gridlines()
base_map.tight_layout()

c:\Users\Joel Efraín\OneDrive - ANDDES ASOCIADOS SAC\Proyectos en curso\_temp\seismic_hazard\.venv\lib\site-packages\cartopy\mpl\feature_artist.py:144: UserWarning: facecolor will have no effect as it has been defined as "never".
  warnings.warn('facecolor will have no effect as it has been '


## 1.5 Datos de salida


In [12]:
# Guardar el mapa de fallas
base_map.save_svg(f"{faults_map}.svg")
base_map.save_png(f"{faults_map}.png")
crop_image(f"{faults_map}.png")

# Guardar tabla de fallas
faults_df.to_csv(f"{disagg_file}.csv", sep=SEP_SYMBOL_COLM, mode="w", index=False)

# 5. Desagregación sísmica


In [13]:
from lib.disaggregation.analyser import DisaggregationAnalyser

import pandas as pd

## 5.1 Datos de entrada


In [14]:
# Leer el archivo CSV
df = pd.read_csv("var/Mag_Dist_Eps-mean-0_111.csv")

In [15]:
# Diccionario con las claves para las columnas X, Y, Z y COLOR
key_dict = {
    "x": "dist",
    "y": "mag",
    "z": "mean",
    "color": "eps",
}

# Color de las barras de desagregación sísmica
colormap_name = "viridis"

# Etiquetas de los ejes
labels = {
    "x_label": "Distancia (km)",
    "y_label": "Magnitud (Mw)",
    "z_label": "Contribución",
}

# Formatos de salida
# formats = ["svg", "png"]
formats = ["svg"]

## 5.2 Inicialización


In [16]:
# Crear ruta de salida del módulo
output_module = f"{output_dir}/05_seismic_disaggregation/"
is_dir(output_module)

# Nombre de archivo de salida
disagg_file = f"{output_module}/05_disagg_table"
disagg_sumz = f"{output_module}/05_disagg_sumz"

In [17]:
# Lista para almacenar todos los valores
disagg_values = []

## 5.3 Procesamiento


In [18]:
for (imt, poe), group in df.groupby(["imt", "poe"]):

    disagg_analyser = DisaggregationAnalyser(
        key_dict=key_dict,
        colormap_name=colormap_name,
        formats=formats,
        labels=labels,
        n_period_return=N_PERIOD_RETURN,
        poe=poe,
        spc_acc=imt,
        df=group,  # Usar el DataFrame del grupo actual
    )
    disagg_analyser.process_data()
    values = disagg_analyser.values

    # Agregar los valores al acumulador
    disagg_values.append(values)

    # Crear nombre de archivos
    name = f"{output_module}/{values['spc_acc']}{SEP_SYMBOL_NAME}{values['tr']}"
    description = disagg_analyser.create_description(**values)

    # Crear gráfico de barras 3D
    disagg_analyser.generate_3d_chart(name, description)

## 5.4 Datos de salida


In [19]:
# Convertir la lista de diccionarios en un DataFrame
disagg_values = pd.DataFrame(disagg_values)

# Guardar el DataFrame en un archivo CSV para referencia futura
disagg_values.to_csv(f"{disagg_file}.csv", sep=SEP_SYMBOL_COLM, mode="w", index=False)